In [5]:
from dotenv import load_dotenv
import pandas as pd
import requests
import json
import csv
import os

load_dotenv()

def get_api_key():
    return os.getenv('ETHEREUM_API_KEY')

def get_connection_url(contract_address):
    return f'https://svc.blockdaemon.com/universal/v1/ethereum/mainnet/account/{contract_address}/txs'


OPTIONS = {
    "headers": {
        "accept": "application/json",
        "X-API-Key": get_api_key()
    }
}

def extract_transaction_hashes(logs):
    return set(log['id'] for log in logs)

def save_hashes_to_csv(hashes, filename):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['hash'])
        for tx_hash in hashes:
            writer.writerow([tx_hash])

def main(ts_start, ts_end, folder_name, contract_address):
    pages = 0
    total_logs_count = 0
    pageToken = None
    merged_logs = []
    
    try:
        if not os.path.exists(f'./data-bd/{folder_name}/'):
            os.makedirs(f'./data-bd/{folder_name}/')

        while True:
            payload = {
                "from": ts_start,
                "to": ts_end,
                "order": 'desc',
                "page_token": pageToken,
                "page_size": 100
            }
        
            response = requests.get(get_connection_url(contract_address), headers=OPTIONS['headers'], params=payload)
            response_json = response.json()

            response_data = response_json['data']
            
            pages += 1
            print(pages)

            if (response_json["total"] != 0):
                merged_logs.extend(response_data)

                if 'meta' not in response_json:
                    break

                pageToken = response_json["meta"]["paging"]["next_page_token"]

                if pageToken == None:
                    break
            
                total_logs_count += response_json["total"]
            else:
                break

        with open(f'./data-bd/{folder_name}/merged_logs.json', 'w') as f:
            json.dump(merged_logs, f)

        print(f"Merged all logs into ./data-bd/{folder_name}/merged_logs.json")
        print(f"Total logs fetched: {total_logs_count}")

        # Extract transaction hashes and save to CSV
        transaction_hashes = extract_transaction_hashes(merged_logs)
        save_hashes_to_csv(transaction_hashes, f'./data-bd/{folder_name}/transaction_hashes.csv')

        print(f"Transaction hashes saved to ./data-bd/{folder_name}/transaction_hashes.csv")
        print(f"Total transaction hashes: {len(transaction_hashes)}")
    except Exception as error:
        print('Error fetching transactions:', error)


**Relevant Timestamps**

1631491200 # September 13, 2021 00:00:00

1640995200 # January 1, 2022 00:00:00

1651156446 # April 28, 2022 15:34:06

1722441775 # July 31, 2024 16:02:55

In [71]:
contract_address = "0x1a2a1c938ce3ec39b6d47113c7955baa9dd454f2"    # QUEREMOS WITHDRAWALS E DEPOSITS

main(1640995200, 1651156446, "logs-01Jan-28Apr", contract_address)

Total transactions: 53600
Merged all logs into ./data/logs-01Jan-28Apr/merged_logs.json
Total logs fetched: 53600
Transaction hashes saved to ./data/logs-01Jan-28Apr/transaction_hashes.csv
Total transaction hashes: 53601


In [82]:
contract_address = "0x64192819ac13ef72bf6b5ae239ac672b43a9af08"    # SO VAMOS QUERER WITHDRAWALS

main(1651156445, 1722441775, "logs-28Apr-today", contract_address)

Merged all logs into ./data/logs-28Apr-today/merged_logs.json
Total logs fetched: 110900
Transaction hashes saved to ./data/logs-28Apr-today/transaction_hashes.csv
Total transaction hashes: 110930


In [2]:
contract_address = "0x1a2a1c938ce3ec39b6d47113c7955baa9dd454f2"    # SO VAMOS QUERER DEPOSITS

main(1631491200, 1640995199, "logs-13Sep-31Dec", contract_address)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## Retrieve Transaction Receipts

In [6]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

RATE_LIMIT = 20  # requests per second
REQUEST_INTERVAL = 1 / RATE_LIMIT

CONNECTION_URL = "https://svc.blockdaemon.com/ethereum/mainnet/native"

def fetch_receipt(tx_hash, folder_name):
    data = {
        "id": 1,
        "jsonrpc": "2.0",
        "method": "eth_getTransactionReceipt",
        "params": [tx_hash]
    }

    try:
        response = requests.post(CONNECTION_URL, headers=OPTIONS['headers'], json=data)
        response_json = response.json()
        if 'result' in response_json:
            return response_json['result']
        else:
            print(f"Error fetching receipt: {response_json}, {tx_hash}")
            return []
    except Exception as e:
        with open(f"./data-bd/{folder_name}/errors.txt", "a") as error_file:
            error_file.write(f"Error retrieving transaction: {tx_hash}, {e}\n")

def process_hashes(hashes, folder_name):
    receipts = []
    total_hashes = len(hashes)
    progress_interval = max(1, total_hashes // 100)  # Update progress every 1%

    with ThreadPoolExecutor(max_workers=RATE_LIMIT) as executor:
        futures = {executor.submit(fetch_receipt, tx_hash, folder_name): tx_hash for tx_hash in hashes}
        
        for idx, future in enumerate(as_completed(futures), 1):
            try:
                result = future.result()
                receipts.append(result)

                if idx % progress_interval == 0 or idx == total_hashes:
                    progress_percentage = (idx / total_hashes) * 100
                    print(f"Progress: {progress_percentage:.2f}% ({idx}/{total_hashes})")
        
                time.sleep(REQUEST_INTERVAL)
            except Exception as e:
                with open(f"./data-bd/{folder_name}/errors.txt", "a") as error_file:
                    error_file.write(f"Error retrieving transaction: {futures[future]}\n")
    return receipts

def retrieve_receipts(folder_name):
    # Read hashes from CSV
    hashes_df = pd.read_csv(f'./data-bd/{folder_name}/transaction_hashes.csv')
    hashes = hashes_df['hash'].drop_duplicates().tolist()

    # Fetch receipts
    receipts = process_hashes(hashes, folder_name)

    # Save receipts to JSON
    with open(f'./data-bd/{folder_name}/tx_receipts.json', 'w') as f:
        json.dump(receipts, f)

    # Print the count of processed receipts
    print(f"Total receipts fetched: {len(receipts)}")


In [8]:
retrieve_receipts("logs-01Jan-28Apr")

Progress: 1.00% (536/53601)
Progress: 2.00% (1072/53601)
Progress: 3.00% (1608/53601)
Progress: 4.00% (2144/53601)
Progress: 5.00% (2680/53601)
Progress: 6.00% (3216/53601)
Progress: 7.00% (3752/53601)
Progress: 8.00% (4288/53601)
Progress: 9.00% (4824/53601)
Progress: 10.00% (5360/53601)
Progress: 11.00% (5896/53601)
Progress: 12.00% (6432/53601)
Progress: 13.00% (6968/53601)
Progress: 14.00% (7504/53601)
Progress: 15.00% (8040/53601)
Progress: 16.00% (8576/53601)
Progress: 17.00% (9112/53601)
Progress: 18.00% (9648/53601)
Progress: 19.00% (10184/53601)
Progress: 20.00% (10720/53601)
Progress: 21.00% (11256/53601)
Progress: 22.00% (11792/53601)
Progress: 23.00% (12328/53601)
Progress: 24.00% (12864/53601)
Progress: 25.00% (13400/53601)
Progress: 26.00% (13936/53601)
Progress: 27.00% (14472/53601)
Progress: 28.00% (15008/53601)
Progress: 29.00% (15544/53601)
Progress: 30.00% (16080/53601)
Progress: 31.00% (16616/53601)
Progress: 32.00% (17152/53601)
Progress: 33.00% (17688/53601)
Progr

In [9]:
retrieve_receipts('logs-28Apr-today')

Progress: 1.00% (1109/110930)
Progress: 2.00% (2218/110930)
Progress: 3.00% (3327/110930)
Progress: 4.00% (4436/110930)
Progress: 5.00% (5545/110930)
Progress: 6.00% (6654/110930)
Progress: 7.00% (7763/110930)
Progress: 8.00% (8872/110930)
Progress: 9.00% (9981/110930)
Progress: 10.00% (11090/110930)
Progress: 11.00% (12199/110930)
Progress: 12.00% (13308/110930)
Progress: 13.00% (14417/110930)
Progress: 14.00% (15526/110930)
Progress: 15.00% (16635/110930)
Progress: 16.00% (17744/110930)
Progress: 17.00% (18853/110930)
Progress: 18.00% (19962/110930)
Progress: 18.99% (21071/110930)
Progress: 19.99% (22180/110930)
Progress: 20.99% (23289/110930)
Progress: 21.99% (24398/110930)
Progress: 22.99% (25507/110930)
Progress: 23.99% (26616/110930)
Progress: 24.99% (27725/110930)
Progress: 25.99% (28834/110930)
Progress: 26.99% (29943/110930)
Progress: 27.99% (31052/110930)
Progress: 28.99% (32161/110930)
Progress: 29.99% (33270/110930)
Progress: 30.99% (34379/110930)
Progress: 31.99% (35488/11

In [4]:
retrieve_receipts('logs-13Sep-31Dec')

Progress: 1.00% (5113/511377)
Progress: 2.00% (10226/511377)
Progress: 3.00% (15339/511377)
Progress: 4.00% (20452/511377)
Progress: 5.00% (25565/511377)
Progress: 6.00% (30678/511377)
Progress: 7.00% (35791/511377)
Progress: 8.00% (40904/511377)
Progress: 9.00% (46017/511377)
Progress: 10.00% (51130/511377)
Progress: 11.00% (56243/511377)
Progress: 12.00% (61356/511377)
Progress: 13.00% (66469/511377)
Error fetching receipt: {'type': 'https://httpproblems.com/http-status/500', 'title': 'Internal Server Error', 'status': 500, 'detail': 'Network connection lost.', 'instance': '/ethereum/mainnet/native', 'trace': {'timestamp': '2024-08-01T23:15:12.748Z', 'requestId': 'a61c5dde-7b87-4c21-aac4-5d4f2086757b', 'buildId': '653e89d5-849a-44a9-bcb5-5e7e8022fd05', 'rayId': '8ac987637cf7343d'}}, 0xc970225ea119caa8f35aa6dd6ec2906ba042397847237c4f97324fb0a103f888
Progress: 14.00% (71582/511377)
Progress: 15.00% (76695/511377)
Progress: 16.00% (81808/511377)
Progress: 17.00% (86921/511377)
Progress:

In [24]:
import os
import re
import json

def load_failed_hashes(filename):
    with open(filename, 'r') as f:
        hashes = [re.findall(pattern='0x[a-fA-F0-9]{64}', string=line)[0] for line in f]
        print(hashes)
    return hashes

def retrieve_failed_receipts(folder_name):
    errors_file = f'./data-bd/{folder_name}/errors.txt'
    if not os.path.exists(errors_file):
        print(f"Error file {errors_file} does not exist.")
        return

    # Load failed transaction hashes from errors.txt
    failed_hashes = load_failed_hashes(f'./data-bd/{folder_name}/errors.txt')

    # Fetch receipts for failed hashes
    failed_receipts = process_hashes(failed_hashes, folder_name)

    # Save failed receipts to JSON
    with open(f'./data-bd/{folder_name}/tx_receipts_2.json', 'w') as f:
        json.dump(failed_receipts, f)

    # Print the count of processed failed receipts
    print(f"Total failed receipts fetched: {len(failed_receipts)}")


In [25]:
retrieve_failed_receipts('logs-28Apr-today')

['0x4da4e925cd13ebe09760cdb61b2865b052d9e0a6d2294bfa9329b528450ea670', '0x5a8b44675a299d13550d8b1acde43f6bd17fd98ab0fed0349d6be511a1824602', '0x464d1b4c0e52af0996f7d7b049e8f3ea2f031ca7f96be1bb8a9e95317cb03156', '0x4bafa2860951c92fea5200692a585f274d1b2a1473bcf43c7afdf5c646aa5739', '0xf8f33a6d80742059fc9e6eefec2f9254f5ebb9279602efb918690a5957bf5e88']
Progress: 20.00% (1/5)
Progress: 40.00% (2/5)
Progress: 60.00% (3/5)
Progress: 80.00% (4/5)
Progress: 100.00% (5/5)
Total failed receipts fetched: 5


In [28]:
retrieve_failed_receipts('logs-01Jan-28Apr')

{'id': 1, 'jsonrpc': '2.0', 'method': 'eth_getTransactionReceipt', 'params': ['0x2c97057f15488be46f4834e254c1b380541a7e3f26e72500b4fb52c3b5419511']}
{'jsonrpc': '2.0', 'id': 1, 'result': {'blockHash': '0x4e5daadc155d9c5d9a17de46371278a2223fbf450c629c71ea7286ac25f3c1a9', 'blockNumber': '0xd864f1', 'contractAddress': None, 'cumulativeGasUsed': '0xe9589c', 'effectiveGasPrice': '0x175b9f24b6', 'from': '0xe09aaf677f463fdbf76404b01965c2b65bfdcf5f', 'gasUsed': '0x27faa', 'logs': [{'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'topics': ['0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c', '0x0000000000000000000000001a2a1c938ce3ec39b6d47113c7955baa9dd454f2'], 'data': '0x00000000000000000000000000000000000000000000000000bfd8b6c1df0000', 'blockNumber': '0xd864f1', 'transactionHash': '0x2c97057f15488be46f4834e254c1b380541a7e3f26e72500b4fb52c3b5419511', 'transactionIndex': '0x90', 'blockHash': '0x4e5daadc155d9c5d9a17de46371278a2223fbf450c629c71ea7286ac25f3c1a9', 'logInd

In [31]:
retrieve_failed_receipts('logs-13Sep-31Dec')

['0xa16af6547c9880fd819cea704dcbc5867352d2d5c4fd421e2b542939608d1737', '0x44c99b714c6cf90035cfc960b5afaec913c04e9aede1750a8f39768c6aeafa65', '0x87ac956747652e8c22a2bfe9c2ff80be87bef702d7f132a4ffd8b22424f49680', '0xfb876566ac0d4f1901ababd71c5aabc0ae9168ea762b5b43c4d14196f3bd4fe3', '0xf48b07fef35b9a6693a3c0dec509692e446a249f458307ba423917e81a0c8a46', '0x4042754fa259ecf720721f3bc417ff2f6252234a018554461bb0e16aec55ed66', '0x189040aa41be909a406ee0179472e0b576e9b0a2a5f8931196fd6cb8677ab20b', '0x85d5d98bf75ad6d502ba72030535364f7a7caf0485013caa87ea07353da35d06', '0x1875910b0be89ce7fa1f8b48047a2ca72c0d6d6976b36621e48e2db5dc3ab2f6', '0x62bf8f70bab466a6d9b15cd2b6611f3fc14a101a351673fe804e5c00bdad09b7', '0x88487d7a209fbf326b0efa820fadcf69989f57d133e1898dc9ee7a3f0a43774b', '0x90b31b63f4a8a1f0c83e702792cbb2f44577d23f0ecdc9507f8a0bd9f52d6101', '0x67735cb09b779e81d18db01351578dd3c44dbcabb3363d76a941c06bfeb62456', '0x1739ad2eb7bbd0d2009425e458cbf67b568551cb5a7df76c82490b3427de4acb', '0x9ddd3fbf7ae8d149

For the second interval, we need to merge the json files with transaction receipts

In [5]:
import json

def merge_json_files(folder_name):
    file1 = f'./data-bd/{folder_name}/tx_receipts.json'
    file2 = f'./data-bd/{folder_name}/tx_receipts_2.json'
    output_file = f'./data-bd/{folder_name}/merged_tx_receipts.json'

    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        json1 = json.load(f1)
        json2 = json.load(f2)

    merged_json = json1 + json2

    with open(output_file, 'w') as f:
        json.dump(merged_json, f)

    print(f"Number of transaction receipts in {file1}: {len(json1)}")
    print(f"Number of transaction receipts in {file2}: {len(json2)}")
    print(f"Number of transaction receipts in the merged file: {len(merged_json)}")


In [35]:
merge_json_files("logs-01Jan-28Apr")

Number of transaction receipts in file1: 53601
Number of transaction receipts in file2: 7
Number of transaction receipts in the merged file: 53608


In [29]:
merge_json_files("logs-28Apr-today")

Number of transaction receipts in ./data/logs-28Apr-today/unique_tx_receipts.json: 110925
Number of transaction receipts in ./data/logs-28Apr-today/tx_receipts_2.json: 5
Number of transaction receipts in the merged file: 110930


In [34]:
merge_json_files("logs-13Sep-31Dec")

Number of transaction receipts in ./data/logs-13Sep-31Dec/tx_receipts.json: 511377
Number of transaction receipts in ./data/logs-13Sep-31Dec/tx_receipts_2.json: 110
Number of transaction receipts in the merged file: 511487


In [6]:
# remove duplicates from json file with result field of receipts
def remove_duplicates(file, folder_name):
    with open(file, 'r') as f:
        receipts = json.load(f)

    unique_receipts = []
    unique_hashes = set()

    for receipt in receipts:
        try:
            tx_hash = receipt.get('transactionHash')
            if tx_hash not in unique_hashes:
                unique_hashes.add(tx_hash)
                unique_receipts.append(receipt)
        except Exception as e:
            print(f"Error processing receipt: {receipt}, {e}")

    # Save unique receipts to a new file
    unique_output_file = f'./data-bd/{folder_name}/unique_tx_receipts.json'
    with open(unique_output_file, 'w') as f:
        json.dump(unique_receipts, f)

    print(f"Unique receipts saved to {unique_output_file}")

    print(f"Removed {len(receipts) - len(unique_receipts)} duplicate receipts")
    print(f"Total receipts: {len(receipts)}")
    print(f"Total unique receipts: {len(unique_receipts)}")


In [7]:
tx_receipts = './data-bd/logs-01Jan-28Apr/merged_tx_receipts.json'

remove_duplicates(tx_receipts, "logs-01Jan-28Apr")

Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Unique receipts saved to ./data-bd/logs-01Jan-28Apr/unique_tx_receipts.json
Removed 7 duplicate receipts
Total receipts: 53608
Total unique receipts: 53601


In [30]:
tx_receipts = './data-bd/logs-28Apr-today/merged_tx_receipts.json'

remove_duplicates(tx_receipts, "logs-28Apr-today")

Unique receipts saved to ./data/logs-28Apr-today/unique_tx_receipts.json
Removed 0 duplicate receipts
Total receipts: 110930
Total unique receipts: 110930


In [39]:
tx_receipts = './data-bd/logs-13Sep-31Dec/merged_tx_receipts.json'

remove_duplicates(tx_receipts, "logs-13Sep-31Dec")

Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object has no attribute 'get'
Error processing receipt: None, 'NoneType' object h

In [4]:
from dotenv import load_dotenv
import pandas as pd
import concurrent.futures
import json
import requests
import os

load_dotenv()

def get_api_key():
    return os.getenv('ETHEREUM_API_KEY')

CONNECTION_URL = "https://svc.blockdaemon.com/ethereum/mainnet/native"

OPTIONS = {
    "headers": {
        "accept": "application/json",
        "X-API-Key": get_api_key()
    }
}

def get_block_data(block_number, errors_file):
    payload = {
        "id": 1,
        "jsonrpc": "2.0",
        "params": [],
        "method": "eth_getBlockByNumber"
    }

    try:
        payload['params'] = [block_number, False]
        response = requests.post(CONNECTION_URL, headers=OPTIONS['headers'], json=payload)

        if response.status_code == 200:
            block = response.json()["result"]
            block_no = int(block["number"], 16)
            timestamp = int(block["timestamp"], 16)
            transactions = len(block["transactions"])
            return f"{block_no},{transactions},{timestamp}\n"
        else:
            with open(errors_file, "a") as error_file:
                error_file.write(f"Error code: {block_number}\n")
            return "null,null,null\n"
    except Exception as e:
        with open(errors_file, "a") as error_file:
            error_file.write(f"Error retrieving block: {block_number}, {e}\n")
        return "null,null,null"

def get_blocks_data(folder_name):
    df = pd.read_csv('./data-bd/logs-13Sep-31Dec/blocks.csv', delimiter=',')
    already_loaded = set(df['block_number'])
    input_file = f'./data-bd/{folder_name}/unique_tx_receipts.json'
    output_file = f'./data-bd/{folder_name}/blocks_2.csv'
    errors_file = f'./data-bd/{folder_name}/errors.txt'

    with open(input_file, 'r') as file:
        tx_receipts = json.load(file)

    print(f"Extracting block number and Unix timestamp from {len(tx_receipts)} transaction receipts...")

    block_numbers = [tx["blockNumber"] for tx in tx_receipts]

    print(f"Extracted {len(block_numbers)} block numbers...")

    with open(output_file, "a") as blocks_file:
        blocks_file.write("block_number,transactions,timestamp\n")
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            # Submit tasks for each block in the range
            futures = {executor.submit(get_block_data, block_number, errors_file): block_number for block_number in block_numbers if block_number is not None and int(block_number, 16) not in already_loaded}

            # Process the completed tasks and write to the file
            for future in concurrent.futures.as_completed(futures):
                block_number = futures[future]
                try:
                    blocks_data = future.result()
                    blocks_file.write(blocks_data)
                except Exception as e:
                    print(f"Error processing block {block_number}: {e}")

    print(f'Extracted block number and Unix timestamp to {output_file}')

In [32]:
get_blocks_data('logs-28Apr-today')

Extracting block number and Unix timestamp from 110930 transaction receipts...
Extracted 110930 block numbers...
Extracted block number and Unix timestamp to ./data/logs-28Apr-today/blocks_2.csv


In [7]:
get_blocks_data('logs-01Jan-28Apr')

Extracting block number and Unix timestamp from 53601 transaction receipts...
Extracted 53601 block numbers...
Extracted block number and Unix timestamp to ./data/logs-01Jan-28Apr/blocks_2.csv


In [8]:
get_blocks_data('logs-13Sep-31Dec')

Extracting block number and Unix timestamp from 511377 transaction receipts...
Extracted 511377 block numbers...
Extracted block number and Unix timestamp to ./data/logs-13Sep-31Dec/blocks_2.csv


In [8]:
def retrieve_missing_blocks(folder_name):
    blocks_file = f'./data-bd/{folder_name}/blocks.csv'
    receipts_file = f'./data-bd/{folder_name}/unique_tx_receipts.json'
    errors_file = f'./data-bd/{folder_name}/errors_2.txt'

    blocks_data = pd.read_csv(blocks_file)

    with open(receipts_file, 'r') as file:
        tx_receipts = json.load(file)

    block_numbers = [tx["blockNumber"] for tx in tx_receipts]
    
    print("Loaded block numbers and block data...")

    print(len(block_numbers))
    print(len(blocks_data['block_number']))
    
    missing_blocks = set(block_numbers) - set(hex(block) for block in blocks_data['block_number'])

    print(f"Total missing blocks: {len(missing_blocks)}")

    with open(blocks_file, 'a') as blocks_file:
        for missing_block in missing_blocks:
            line = get_block_data(missing_block, errors_file)
            blocks_file.write(line)

In [30]:
retrieve_missing_blocks('logs-01Jan-28Apr')

Loaded block numbers and block data...
5022
5022
Total missing blocks: 0


In [9]:
retrieve_missing_blocks('logs-28Apr-today')

Loaded block numbers and block data...
110930
110925
Total missing blocks: 5


## Filter events of interest based on additional data (only withdrawals or deposits)

Contract of interest:
- 0x64192819ac13ef72bf6b5ae239ac672b43a9af08

topic 0:
- 0xd7b25068d9dc8d00765254cfb7f5070f98d263c8d68931d937c7362fa738048b // DepositRequested (bytes32 receiptHash, tuple receipt)
- 0x21e88e956aa3e086f6388e899965cef814688f99ad8bb29b08d396571016372d // Withdrew (bytes32 receiptHash, tuple receipt)

### 

Contract of interest:
- 0x1a2a1c938ce3ec39b6d47113c7955baa9dd454f2

topic 0:
- 0x86174ea401f083b9bb1bdebca3068f27fb023c7091365ed2a8a02b8d75cf0e52 // TokenWithdrew (index_topic_1 uint256 _withdrawId, index_topic_2 address _owner, index_topic_3 address _tokenAddress, uint256 _tokenNumber)
- 0x72848855a2461abf0dd243723dfcc9163eec2ea5215469d101c0d9c9ef58940d // TokenDeposited (index_topic_1 uint256 _depositId, index_topic_2 address _owner, index_topic_3 address _tokenAddress, address _sidechainAddress, uint32 _standard, uint256 _tokenNumber)


In [7]:
import json

def retrieve_withdrawals(input_file, output_file, contract_address, topic_0):
    withdrawal_receipts = []

    with open(input_file, 'r') as infile:
        json_data = json.load(infile)
        print("Total tx receipts: ", len(json_data))

    for receipt in json_data:
        for log in receipt['logs']:
            if log['address'] == contract_address and log['topics'][0] == topic_0:
                withdrawal_receipts.append(receipt)

    with open(output_file, 'w') as outfile:
        json.dump(withdrawal_receipts, outfile)

    print("Total withdrawal txs: ", len(withdrawal_receipts))

In [8]:
input_file = f"./data-bd/logs-28Apr-today/unique_tx_receipts.json"
output_file = f"./data-bd/logs-28Apr-today/withdrawal_receipts.json"

retrieve_withdrawals(input_file, output_file, "0x64192819ac13ef72bf6b5ae239ac672b43a9af08", "0x21e88e956aa3e086f6388e899965cef814688f99ad8bb29b08d396571016372d")

Total tx receipts:  110930
Total withdrawal txs:  19482


In [9]:
input_file = f"./data-bd/logs-13Sep-31Dec/unique_tx_receipts.json"
output_file = f"./data-bd/logs-13Sep-31Dec/deposit_receipts.json"

retrieve_withdrawals(input_file, output_file, "0x1a2a1c938ce3ec39b6d47113c7955baa9dd454f2", "0x72848855a2461abf0dd243723dfcc9163eec2ea5215469d101c0d9c9ef58940d")

Total tx receipts:  511377
Total withdrawal txs:  459299
